In [51]:
# Dependencies
import numpy as np
import pandas as pd
import datetime as dt

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [3]:
engine = create_engine("sqlite:///hawaii.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [4]:
# Create our session (link) from Python to the DB
session = Session(engine)

## D1: Determine the Summary Statistics for June

In [132]:
# 1. Import the sqlalchemy extract function.
from sqlalchemy import extract

# 2. Write a query that filters the Measurement table to retrieve the temperatures for the month of June. 
june_observations = session.query(Measurement.tobs).filter(Measurement.date.contains('-06-')).all()
june_observations[0:5]

[(78.0), (76.0), (78.0), (76.0), (77.0)]

In [135]:
# 3. Convert the June temperatures to a list.
june_temps = [temp[0] for temp in june_observations]
june_temps[0:5]
type(june_temps)

[78.0, 76.0, 78.0, 76.0, 77.0]

list

In [141]:
# 4. Create a DataFrame from the list of temperatures for the month of June. 
june_df = pd.DataFrame(june_temps, columns=['June Temps'])
june_df.head()

,June Temps
0,78.0
1,76.0
2,78.0
3,76.0
4,77.0


In [142]:
# 5. Calculate and print out the summary statistics for the June temperature DataFrame.
june_df.describe()

,June Temps
count,1700.000000
mean,74.944118
std,3.257417
min,64.000000
25%,73.000000
50%,75.000000
75%,77.000000
max,85.000000


## D2: Determine the Summary Statistics for December

In [127]:
# 6. Write a query that filters the Measurement table to retrieve the temperatures for the month of December.
dec_observations = session.query(Measurement.tobs, Measurement.date).filter(Measurement.date.contains('-12-')).all()
dec_observations[0:5]

[(76.0, '2010-12-01'),
 (74.0, '2010-12-03'),
 (74.0, '2010-12-04'),
 (64.0, '2010-12-06'),
 (64.0, '2010-12-07')]

In [131]:
# 7. Convert the December temperatures to a list.
dec_temps = [temp[0] for temp in dec_observations]
dec_temps[0:5]
type(dec_temps)

[76.0, 74.0, 74.0, 64.0, 64.0]

list

In [143]:
# 8. Create a DataFrame from the list of temperatures for the month of December. 
dec_df = pd.DataFrame(dec_temps, columns=['December Temps'])
dec_df.head()

,December Temps
0,76.0
1,74.0
2,74.0
3,64.0
4,64.0


In [144]:
# 9. Calculate and print out the summary statistics for the Decemeber temperature DataFrame.
dec_df.describe()

,December Temps
count,1517.000000
mean,71.041529
std,3.745920
min,56.000000
25%,69.000000
50%,71.000000
75%,74.000000
max,83.000000


-------

### Additonal Analysis 

In [186]:
# days in june that are higher than the average 
june_high = [i for i in june_df['June Temps'] if i > 75.00]
december_high_avg = [i for i in dec_df['December Temps'] if i > 71.00]

In [188]:
len(june_high)

791

In [179]:
round(len(june_high_avg) / len(june_df) * 100,2)
round(len(december_high_avg) / len(dec_df) * 100,2)

46.53

47.0

In [193]:
# days in june and december with recorded temperatures higher than the average 
june_high_avg = session.query(Measurement.tobs).filter(Measurement.tobs > 75, Measurement.date.contains('-06-')).all()
dec_high_avg = session.query(Measurement.tobs).filter(Measurement.tobs > 71, Measurement.date.contains('-12-')).all()

In [304]:
# percentage of observations above the averge temperature observation 
round(len(june_high_avg) / len(june_df) * 100, 2)
round(len(dec_high_avg) / len(dec_df) * 100, 2)

46.53

47.0

In [297]:
cat = [50,60,70,80,90,100]
june_df['Temp Range'] = pd.cut(june_df['June Temps'],cat)

In [298]:
june_temp_range_count = june_df.groupby('Temp Range').count().reset_index()
june_temp_range_count.rename(columns={'June Temps':'Observation Count'})

,Temp Range,Observation Count
0,"(50, 60]",0
1,"(60, 70]",171
2,"(70, 80]",1485
3,"(80, 90]",44
4,"(90, 100]",0


In [299]:
cat = [50,60,70,80,90,100]
dec_df['Temp Range'] = pd.cut(dec_df['December Temps'],cat)

In [300]:
dec_temp_range_count = dec_df.groupby('Temp Range').count().reset_index()
dec_temp_range_count.rename(columns={'December Temps':'Observation Count'})

,Temp Range,Observation Count
0,"(50, 60]",9
1,"(60, 70]",621
2,"(70, 80]",883
3,"(80, 90]",4
4,"(90, 100]",0
